# Few-shot JERX task offline inference

In [ ]:
#|default_exp jerx.offline.llm

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
import json
from pathlib import Path
from bellem.jerx.utils import parse_triplets
from bellem.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#|export

def parse_triplet_response(response: str, *args, **kwargs) -> list[tuple[str, str, str]]:
    triplets = parse_triplets(response.strip())
    return [(e1, rel, e2) if e1 != e2 else (e1, rel, e2 + "(obj)") for e1, rel, e2 in triplets]


def make_kg_triplet_extract_fn(inference_cache_filepath: Path):
    mapping = {}
    with open(inference_cache_filepath, "r") as f:
        for i, line in enumerate(f):
            record = json.loads(line.strip())
            mapping[record["text"]] = record["generation"]

    def extract_kg_triplets(text: str) -> list[tuple[str, str, str]]:
        return parse_triplet_response(mapping[text])

    return extract_kg_triplets

In [ ]:
#|hide
jerx_inferences_filepath = Path("../data/raw/musique-evaluation/jerx-inferences/llama3-base.jsonl")
func = make_kg_triplet_extract_fn(jerx_inferences_filepath)

In [ ]:
#|hide
text = "# Maria Carrillo High School\nMaria Carrillo High School is a public high school located in Santa Rosa, California, United States. It is managed by the Santa Rosa City Schools district. It opened in 1996 and is located in the Rincon Valley neighborhood of Santa Rosa."
print(func(text))

[('Maria Carrillo High School', 'location', 'Santa Rosa, California, United States'), ('Maria Carrillo High School', 'management', 'Santa Rosa City Schools district'), ('Maria Carrillo High School', 'opening year', '1996')]


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()